In [1]:
import numpy as np


import matplotlib
matplotlib.use('nbAgg')
import matplotlib.pyplot as plt

In [2]:
class modelFunc:
    def __init__(self,func,w0,dim_w,dim_x,dim_fx):
        self.func=func
        self.w=w0
        self.dim_w=dim_w
        self.dim_x=dim_x
        self.dim_fx=dim_fx
        
        
        self.J=[[0 for _1 in range(self.dim_x)] for _2 in range(self.dim_fx)]
        
        
    def __call__(self,x):
        return self.func(self,x)
    
    def Jacobian(self,x,h=1e-5):
        x0=x[:]
        x1=x[:]
        for i in range(self.dim_fx):
            for j in range(self.dim_x):
                x0[j]=x[j]-h
                x1[j]=x[j]+h
                
                self.J[i][j]=(self(x1)[i]-self(x0)[i])/(2*h)
        
                x0[j]=x[j]
                x1[j]=x[j]
        


In [3]:
class DifferentialEquation:
    def __init__(self,model,rhs,x0,fx0,ranges):
        self.model=model
        self.rhs=rhs
        self.x0=x0
        self.fx0=fx0
        self.ranges=ranges
        
        self.grad=[0 for i in  range(self.model.dim_w)]
        
    def RHS(self,x):
        return self.rhs(self.model,x)
    
    def loss(self,x):
        Nx=self.model.dim_fx*self.model.dim_x
        
        Qx=0        
        self.model.Jacobian(x)
        RHS=self.RHS(x)
        
        for i in range(self.model.dim_fx):
            for j in range(self.model.dim_x):
                Qx+=(self.model.J[i][j]-RHS[i][j])**2
        
        N0=self.model.dim_fx
        Qx0=0
        mx0=self.model(self.x0)
        for i,fx0 in enumerate(self.fx0):
            Qx0+=(fx0 - mx0[i])**2

        
        Q=Qx/Nx+Qx0/N0
        return Q/2.
    
    
    def lossGrad(self,x,h=1e-5):
        
        for dim in range(self.model.dim_w):

            self.model.w[dim]-=h
            Q0=self.loss(x)

            self.model.w[dim]+=2*h
            Q1=self.loss(x) 
            
            self.model.w[dim]-=h

            self.grad[dim]=(Q1-Q0)/(2*h)

    
    
    def update(self,alpha=1e-2,abs_tol=1e-5, rel_tol=1e-3):
        x=[np.random.rand()*(r[1]-r[0])+r[0] for r in self.ranges]      
        
        self.lossGrad(x)
        _w2=0
        _check=0
        for i,g in enumerate(self.grad):
            self.model.w[i]+=-alpha*g
            
            _w2=abs_tol + self.model.w[i] * rel_tol
            _check+=(g/_w2)*(g/_w2)

        _check=np.sqrt(1./self.model.dim_w *_check)
        
        
        
        return _check
        
        
        
    def run(self,alpha=1e-2,abs_tol=1e-5, rel_tol=1e-3, step_break=100,max_step=5000):
        '''        
        abs_tol, rel_tol, step_break: stop when _check<1 (_check is what update should return) 
        for step_break consecutive steps
        
        max_step: maximum number of steps
        '''
        _s=0
        count_steps=1
        while count_steps<=max_step:
            _check=self.update(alpha,abs_tol, rel_tol)
            
            count_steps+=1             
                
            
            if _check<1:
                _s+=1
            else:
                _s=0
            
            if _s>step_break:
                break

        return self.model.w[:]

In [4]:
def func(self,x):
    return [self.w[0]*np.exp(-self.w[1]*x[0]),self.w[2]*x[0]**2+self.w[3]]

def RHS(func,x):
    '''
    df1dx1=-2*f1(x1)
    df2dx1=2*x1
    '''
    fx=func(x)
    return [[-2*fx[0]],[2*x[0]]]

In [5]:
Model=modelFunc(
    func=func,
    w0=[0,1,-1,0],
    dim_w=4,
    dim_x=1,
    dim_fx=2)

DE=DifferentialEquation(Model,RHS,[0],[1,-1],ranges=[[0,1]])

In [6]:
DE.run()

[0.9999761687201554,
 1.9925072279997103,
 1.0000000000029117,
 -0.9999999988591718]